In [1]:
from copy import deepcopy
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from src.jss_lite.jss_lite import jss_lite
import numpy as np
import time
class tree_node:
    def __init__(self, action=None,parent=None):
        self.action=action
        self.children=[]
        self.state=None
        self.parent=parent
        self.observation=None
        self.visited=False
        # extra attributes for search:
        self.leaf_closed=False
class tree_construct:
    def __init__(self):

        curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
        inst=curr_dir + '/resources/jsp_instances/standard/la01.txt'
        instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
        self.env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
        # create root
        self.root=tree_node()
        # get initial state into root; todo: change for multi instances, create all childs for instances
        self.root.observation=self.env.reset()
        self.root.state=self.env.get_state()
        self.done=False
        self.reward=0

        # path management
        # open leafs should store the leafs with highest rating
        self.open_leafs=[]
        self.open_leafs.append(self.root)
        self.current_path=self.root
        self.sol_pathes=None
    def insert(self,action=None,parent=None):
        parent.children.append(tree_node(action=action,parent=self))
        self.env.set_state(parent.state)
        parent.children[-1].observation,reward,done,info=self.env.step(action)
        parent.children[-1].state=self.env.get_state()
        if done:
            self.__on_done(reward)

    def __on_done(self,reward):
        self.done=True
        if reward > self.reward:
            self.reward=reward

    def solve(self,max_pathes=None,max_time=None,threshold=0.4):
        # here comes the solving method with abort conditions
        start_time=time.time()
        p_time=time.time()
        self.sol_pathes=0

        if max_pathes==None:
            max_pathes=1000 
        if max_time==None:
            max_time=60
        #print(self.open_leafs)
        while self.open_leafs:
            self.sol_pathes+=1
            # abort conditions:
            node=self.open_leafs.pop()
            if self.sol_pathes > max_pathes or time.time()-start_time > max_time:
                return False
            while not self.done:
                loop=0
                n_r=np.random.uniform(0,1)
                if  n_r < 0.05:
                    #print(3)
                    p_r=3
                elif n_r < 0.1:
                    #print(2)
                    p_r=2
                else:
                    p_r=1
                possible_actions=np.random.choice(len(node.observation['action_mask']), p_r, p=(node.observation['action_mask'] / node.observation['action_mask'].sum()))
                
                for action in set(possible_actions):
                    self.insert(action=action,parent=node)
                for child in node.children:
                    if child != node.children[-1]:
                        #todo: check here if in action mask
                        self.open_leafs.append(child)
                while True:
                    if node.children[-1].visited==False:
                        node.children[-1].visited=True
                        node=node.children[-1]
                        
                        break
                    loop+=1
                    if loop > len(node.children):
                        print("all childrens visited; return done equal true")
                        self.done=True
            self.done=False
            




In [2]:
init=tree_construct()
init.solve()
print(init.reward)
print(init.sol_pathes)

0.924924924924925
64


In [3]:

from ray.rllib.agents.ppo import PPOTrainer
from ray import tune
import gym
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_no_action_mask
from src.jss_lite.jss_lite import jss_lite
from ray import tune
curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
inst=curr_dir + '/resources/jsp_instances/standard/la01.txt'
instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
# Configure the algorithm.
def env_creator(env_config):
    #env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
    env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))
    return env

# use tune to register the custom environment for the ppo trainer
tune.register_env('custom_jssp',env_creator)


config = {
    # Environment (RLlib understands openAI gym registered strings).
    #"env": "CartPole-v1",
    "env": "custom_jssp",
    "disable_env_checking":True,

    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 4,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "tf",
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    #"horizon":1,
    #"evaluation_duration":0,
    "model": {
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 0,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": False,
    },
    
}

trainer = PPOTrainer(config=config)

for _ in range(4):
    print(_)
    trainer.train()




2022-10-26 15:53:20,609	INFO trainable.py:159 -- Trainable.setup took 14.540 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


0


2022-10-26 15:53:22,111	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


1
2
3


In [48]:
trainer.save_checkpoint("ppo")
episode_reward = 0
#env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))
done = False
obs = env.reset()
iterations=0


while not done:
    policy = trainer.get_policy()
    action, _, info = policy.compute_single_action(obs)
    logits = info['action_dist_inputs']
    action_mask = env.get_legal_actions().astype(np.float32)
    action_mask[action_mask==0] = - np.inf
    probs=np.multiply(logits,action_mask)
    probs[probs==np.inf]=-np.inf
    best_action_id = probs.argmax()  # deterministic

    obs, reward, done, info = env.step(best_action_id)
    episode_reward += reward
    iterations += 1
    print(best_action_id)
    print(action_mask[best_action_id])
    if action_mask[best_action_id] ==-np.inf:
        break
    env.render()
print(episode_reward)
print(iterations)

KeyboardInterrupt: 

In [49]:
env.render()

In [ ]:
print(info)
print(len(info['action_dist_inputs']))

In [ ]:
import numpy as np
import time
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from src.jss_lite.jss_lite import jss_lite
curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
inst=curr_dir + '/resources/jsp_instances/standard/ft06.txt'
instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
# Configure the algorithm.
def env_creator(env_config):
    env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
    #env=jss_lite(instance_path=inst)
    return env
# get random actions:
reward=0
while reward <0.99:
#for _ in range(100):
    action_list=[]
    env = env_creator("s")

    obs = env.reset()
    # env2 is copy for later going evaluation

    done = False
    t=time.time()
    while not done:
        legal_action=obs['action_mask']
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        action_list.append(action)
        #print(action)
        #print(obs['action_mask'][action])
        obs, reward, done, _ = env.step(action)
    #print(f"instance: {env.env.instance} in time: {time.time()-t}s")
    
    #env.render()
    #env.close()
print(reward)
env.render()